In [1]:
import webbrowser
import pandas as pd
from googleapiclient.discovery import build

pd.options.display.max_colwidth = 100

api_key = 'AIgbSyDd4XAIGtFGcvbnflN7u52biHmBJyStL42'

In [2]:
search_term = "FastAPI"  # Don't use '-' or '|' in the search_term

duration = "any"  # any OR long(>20) OR medium(4-20) OR short(<4)

published_after = "2020-01-01"  # "%Y-%m-%d"

In [3]:
youtube = build("youtube", "v3", developerKey=api_key)

next_page_token = None
titles, urls, stats, dates = [], [], [], []

for _ in range(2):
    res = (
        youtube.search()
        .list(
            q=search_term,
            part="snippet",
            type="video",
            order="relevance",
            maxResults=50,
            safeSearch="none",
            publishedAfter=published_after + "T00:00:00Z",
            videoDuration=duration,
            pageToken=next_page_token,
        )
        .execute()
    )
    items = res["items"]
    titles += [i["snippet"]["title"] for i in items]
    dates += [i["snippet"]["publishedAt"][:10] for i in items]
    video_ids = [i["id"]["videoId"] for i in items]
    urls += [f"https://www.youtube.com/watch?v={i}" for i in video_ids]
    stats += [i["statistics"] for i in youtube.videos().list(id=",".join(video_ids), part="statistics").execute()["items"]]

    next_page_token = res.get("nextPageToken")

views = [int(i["viewCount"]) for i in stats]
likes = [int(i.get("likeCount", 0)) for i in stats]

data = {"Title": titles, "Views": views, "Likes": likes, "Date": dates, "URL": urls}
df = pd.DataFrame(data)
df.insert(3, "L/V", (df["Likes"] / df["Views"]).round(4))

In [4]:
df1 = df[df["Views"] > df["Views"].quantile(0.85)].sort_values("L/V", ascending=False)
df1.head(10)

,Title,Views,Likes,L/V,Date,URL
7,FastAPI Tutorial - Building RESTful APIs with Python,48294,2208,0.0457,2021-11-08,https://www.youtube.com/watch?v=GN6ICac3OXY
3,Fast API crash course | easy way,45992,1743,0.0379,2021-01-28,https://www.youtube.com/watch?v=TQfIUS52QHA
5,Python API Development - Comprehensive Course for Beginners,714058,25451,0.0356,2021-11-01,https://www.youtube.com/watch?v=0sOvCWFmrtA
4,Python FAST API Tutorial,108186,3520,0.0325,2021-05-29,https://www.youtube.com/watch?v=-ykeT6kk4bk
6,I built the same app 3 times | Which Python Framework is best? Django vs Flask vs FastAPI,105879,3259,0.0308,2022-02-20,https://www.youtube.com/watch?v=3vfum74ggHE
22,Deploying FastAPI Apps Over HTTPS with Traefik Proxy,25457,773,0.0304,2021-02-25,https://www.youtube.com/watch?v=7N5O62FjGDc
2,FastAPI Course for Beginners,108283,2826,0.0261,2021-08-12,https://www.youtube.com/watch?v=tLKKmouUams
9,FastAPI Introduction - Build Your First Web App - Python Tutorial,26000,664,0.0255,2020-11-04,https://www.youtube.com/watch?v=0RS9W8MtZe4
13,"Let&#39;s Build a Fast, Modern Python API with FastAPI",46225,1075,0.0233,2020-12-15,https://www.youtube.com/watch?v=sBVb4IB3O_U
10,Intro to FastAPI - The Best Way to Create APIs in Python?,94927,2179,0.0230,2020-09-04,https://www.youtube.com/watch?v=kCggyi_7pHg


In [5]:
df2 = df[(df["Views"].quantile(0.85) > df["Views"]) & (df["Views"] >= df["Views"].quantile(0.60))].sort_values("L/V", ascending=False)
df2.head(10)

,Title,Views,Likes,L/V,Date,URL
66,Python &amp; FastAPI Tutorial: Create an ai microservice to extract text from images.,17578,764,0.0435,2021-09-04,https://www.youtube.com/watch?v=JxH7cdDCFwE
19,How to serve any machine learning or deep learning model using FastAPI,13614,566,0.0416,2021-01-24,https://www.youtube.com/watch?v=Mw9etoRz0Ic
23,"FARM Stack Course ( FastAPI , React js , mongoDB ) in 1 Hour [Presented in FreeCodeCamp as well]",16240,581,0.0358,2021-06-16,https://www.youtube.com/watch?v=oGwszbCH5Z4
79,Build a Blockchain with Python &amp; FastAPI,9205,329,0.0357,2021-09-28,https://www.youtube.com/watch?v=G5M4bsxR-7E
46,REST API IN PYTHON | Flask vs. FastAPI vs. Django,11349,372,0.0328,2021-08-28,https://www.youtube.com/watch?v=0juyVjBN8HQ
14,Modern Python through FastAPI and friends. Sebastián Ramírez.,9323,299,0.0321,2021-09-19,https://www.youtube.com/watch?v=37CcB2GBdlY
17,FastAPI [ Python Web Framework ] Crash Course 2021 For Beginners,15681,500,0.0319,2021-01-31,https://www.youtube.com/watch?v=62pP9pfzNRs
1,FastAPI Tutorial | FastAPI vs Flask,20162,602,0.0299,2021-08-18,https://www.youtube.com/watch?v=Wr1JjhTt1Xg
18,FastAPI - Build your first API with Python&#39;s Fastest API Web Framework!,19992,590,0.0295,2021-06-29,https://www.youtube.com/watch?v=NH4VZaP3_9s
73,"Create A REST API with FastAPI, SQLAlchemy and PostgreSQL.",16481,481,0.0292,2021-06-24,https://www.youtube.com/watch?v=2g1ZjA6zHRo


In [6]:
df3 = df[(df["Views"].quantile(0.60) > df["Views"]) & (df["Views"] >= df["Views"].quantile(0.30))].sort_values("L/V", ascending=False)
df3.head(10)

,Title,Views,Likes,L/V,Date,URL
16,Why is FastAPI taking off? Here&#39;s one reason,7661,410,0.0535,2021-08-03,https://www.youtube.com/watch?v=jaYsrGpig-s
36,SQLModel: The New ORM for FastAPI and Beyond - Talk Python Podcast Episode,3606,157,0.0435,2022-01-18,https://www.youtube.com/watch?v=BTCtKWpNoRI
56,DEPLOY FastAPI with Airtable to Heroku with Github!,7448,307,0.0412,2021-01-27,https://www.youtube.com/watch?v=EZbXvn5CwpM
12,FastAPI Microservices Full Course,4904,193,0.0394,2022-03-15,https://www.youtube.com/watch?v=jJYHUp9ZeTY
74,Smarter FastAPI Through Tooling,6779,251,0.0370,2021-10-28,https://www.youtube.com/watch?v=r6fBoBTRfT4
39,GraphQL and FastAPI integration,5169,156,0.0302,2021-08-01,https://www.youtube.com/watch?v=2_puWfTK8bQ
33,FastAPI and GraphQL - build your first API,8129,233,0.0287,2021-07-05,https://www.youtube.com/watch?v=Puvr82Cm26o
87,"Deploy with FastAPI | | Sentiment Analysis with BERT using huggingface, PyTorch and Python Tutorial",8449,230,0.0272,2020-05-02,https://www.youtube.com/watch?v=K4rRyAIn0R0
40,Microservices with FastAPI and Docker (Python based) - Hands on Tutorial,2786,75,0.0269,2021-09-20,https://www.youtube.com/watch?v=cTTTC3IhkDc
78,"Web API with FastAPI, RabbitMQ and Celery",2824,68,0.0241,2021-05-17,https://www.youtube.com/watch?v=a0ODIWsCgDI


In [7]:
break

SyntaxError: 'break' outside loop (Temp/ipykernel_12008/668683560.py, line 1)

In [ ]:
for i in df1.URL[:5]:
    webbrowser.open_new_tab(i)

In [ ]:
for i in df2.URL[:5]:
    webbrowser.open_new_tab(i)

In [ ]:
for i in df3.URL[:5]:
    webbrowser.open_new_tab(i)